In [2]:
!curl https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py > minsearch.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3832  100  3832    0     0  22022      0 --:--:-- --:--:-- --:--:-- 22022


In [3]:
!ls -ltrh

total 208K
-rw-rw-rw-  1 codespace root        14 Jun 28 18:06 README.md
drwxrwxrwx+ 2 codespace codespace 4.0K Jul  1 02:54 faq_documents
-rw-rw-rw-  1 codespace codespace 189K Jul  4 04:27 rough-note.ipynb
-rw-rw-rw-  1 codespace codespace  337 Jul  4 04:37 rag-intro.ipynb
-rw-rw-rw-  1 codespace codespace 3.8K Jul  4 04:38 minsearch.py


In [12]:
import minsearch

In [7]:
!curl https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json > documents.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


In [2]:
import requests 
import json

In [3]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

with open('./faq_documents/documents.json', 'w+') as f_json:
    f_json.write(json.dumps(documents_raw, indent=4))

In [13]:
with open('./faq_documents/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
type(docs_raw)

list

In [14]:
documents = []

In [15]:
for document_dict in docs_raw:
    for each_document in document_dict['documents']:
        each_document['course'] = document_dict['course']
        documents.append(each_document)

In [16]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
q = 'the course is already started. can I still enroll?'

In [9]:
index.fit(documents)

In [10]:
boost = {
    'question': 3.0,
    'text': 0.5
}

results = index.search(
    q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [10]:
from openai import OpenAI

In [11]:
client = OpenAI()

In [14]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[{"role": "user", "content": q}]
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [36]:
import os
HF_API_TOKEN = os.getenv('HF_API_TOKEN')

In [14]:
API_URL = "https://api-inference.huggingface.co/models/gpt2"
headers = {"Authorization": f"Bearer {HF_API_TOKEN}"}
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

In [17]:
response = requests.post(API_URL, headers=headers, json=q)

In [20]:
payload={
    "inputs": "the course is already started. can I still enroll?"
}

In [21]:
response = requests.post(API_URL, headers=headers, json=payload)

In [25]:
response.json()[0]['generated_text']

'the course is already started. can I still enroll? 69% * Please complete the application if you already have a valid university ID and have at least $1,000 prior history. AP Enrollment Calculator 14-2016 Minimum $54 minimum fine Minimum $6 for whole week 13-17 $4\n\nJUDE (BA; berlin, U.S.)\n\n\n= INQUISITRATION CHAT WITH 162 GENERAL RESEARCH ASIDE COMMITMENT- Entrance to an Early Degree is $7.78 per requirement'

In [48]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain the answer, output NONE.

QUESTION: {question}

CONTEXT:
{context}
"""

In [49]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [14]:
context

"section: General course-related questions\nquestion: Course - Can I still join the course after the start date?\nanswer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n\nsection: General course-related questions\nquestion: Course - Can I follow the course after it finishes?\nanswer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.\n\nsection: General course-related questions\nquestion: Course - What can I do before the course starts?\nanswer: You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with A

In [15]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - What can I do before the course starts?
answer: You can start by installing and setting up all the dependencies and requirements:
Google cloud account
Google Cloud SDK
Python 3 (installed with Anaconda)
Terrafo

In [50]:
prompt = prompt_template.format(question=q, context=context).strip()

In [42]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain the answer, output NONE.
Print only the ANSWER, Do not add any other terms in the output.

QUESTION: the course is already started. can I still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing f

In [34]:
from huggingface_hub import InferenceClient

In [18]:
client = InferenceClient()

In [22]:
response = client.chat_completion(
    prompt=prompt,
    model="codellama/CodeLlama-7b-hf",
    temperature=0.8,
    max_new_tokens=50,
    seed=42,
    return_full_text=True,
)

In [23]:
print(response)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain the answer, output NONE.

QUESTION: the course is already started. can I still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fi

In [51]:
messages = [{"role": "user", "content": prompt}]

In [37]:
client = InferenceClient("HuggingFaceH4/zephyr-7b-beta", token=HF_API_TOKEN)

In [53]:
response = client.chat_completion(messages, max_tokens=100)

In [54]:
print(response.choices[0].message.content)

QUESTION: The course is already started. Can I still enroll?

ANSWER: Yes, according to the context, even if you miss the registration deadline, you're still eligible to submit the homeworks. However, you need to be aware that there will be deadlines for turning in the final projects, so it's recommended not to leave everything for the last minute. If you'd like to follow the course after it finishes or join the Telegram


In [27]:
def search(query):
    boost = {
        'question': 3.0,
        'text': 0.5
    }
    
    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [28]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain the answer, output NONE.

QUESTION: {question}

CONTEXT:
{context}
"""

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [40]:
def llm(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat_completion(messages, max_tokens=300)
    return response.choices[0].message.content

In [30]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [76]:
rag('how do I run kafka')

'To run Kafka in Java, you should follow the steps provided in the context under the section "Module 6: streaming with Kafka". In the project directory, run the following command in the terminal:\n\n`java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`\n\nReplace `<jar_name>` with the name'

In [77]:
rag('The course has already started. Can I still enroll?')

'Based on the context provided, here\'s the answer to the question:\n\nQUESTION: The course has already started. Can I still enroll?\n\nANSWER: Based on the "Course - Can I still join the course after the start date?" question and answer, it\'s clear that even though the course has already started, it\'s still possible to enroll. However, as mentioned in the "Course - Can I follow the course after it'

In [5]:
from elasticsearch import Elasticsearch

In [6]:
es_client = Elasticsearch('http://localhost:9200')

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name  = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [10]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:22<00:00, 41.46it/s]


In [18]:
query = "the course is already started. can I still enroll?"

In [25]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs =  []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [20]:
response = es_client.search(index=index_name, body=search_query)

In [23]:
result_docs =  []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [26]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [31]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [41]:
print(rag(query))

QUESTION: Can I still enroll in the course after it has already started?

ANSWER: Yes, according to the given context, even if you don't register, you're still eligible to submit homeworks, but be aware that there will be deadlines for turning in the final projects, so don't leave everything for the last minute. It's recommended to also review the prerequisites and syllabus before starting to ensure comfort with the required subjects. The course materials will be kept after it finishes, allowing self-paced learning after it has ended, as well as the ability to continue preparing for future cohorts or start working on the final capstone project. No specific timeline for confirmation email was mentioned, however, registration is optional and not required to start learning or submitting homework.
